# 4章 形態素解析
MeCabで形態素解析する
./data/neko.txt → ./data/neko.text.mecab

## 30. 形態素解析結果の読み込み
neko.txt.mecabを読み込みプログラムを作成する。
各形態素は表層形(surface)、基本形(base)、品詞(pos)、品詞細分類1(pos1)をキーとするハッシュ型とする。1文はこのリストとする。

In [1]:
DATA_PATH = "./data/neko.txt.mecab"

In [8]:
def create_datalist():
    result_list = []
    word_info_list = []
    with open(DATA_PATH) as f:
        for line in f:
            if line != "EOS\n":
                groups = line.split('\t')
                if len(groups) == 2 and groups[0] != '':
                    details = groups[1].split(',')
                    word_info = {"surface": groups[0], "base": details[6], "pos": details[0], "pos1": details[1]}
                    word_info_list.append(word_info)
            else:
                result_list.append(word_info_list)
                word_info_list = []
    return result_list

In [12]:
data_list = create_datalist()

for w in data_list[5]:
    print(w)

{'surface': '\u3000', 'base': '\u3000', 'pos': '記号', 'pos1': '空白'}
{'surface': 'どこ', 'base': 'どこ', 'pos': '名詞', 'pos1': '代名詞'}
{'surface': 'で', 'base': 'で', 'pos': '助詞', 'pos1': '格助詞'}
{'surface': '生れ', 'base': '生れる', 'pos': '動詞', 'pos1': '自立'}
{'surface': 'た', 'base': 'た', 'pos': '助動詞', 'pos1': '*'}
{'surface': 'か', 'base': 'か', 'pos': '助詞', 'pos1': '副助詞／並立助詞／終助詞'}
{'surface': 'とんと', 'base': 'とんと', 'pos': '副詞', 'pos1': '一般'}
{'surface': '見当', 'base': '見当', 'pos': '名詞', 'pos1': 'サ変接続'}
{'surface': 'が', 'base': 'が', 'pos': '助詞', 'pos1': '格助詞'}
{'surface': 'つか', 'base': 'つく', 'pos': '動詞', 'pos1': '自立'}
{'surface': 'ぬ', 'base': 'ぬ', 'pos': '助動詞', 'pos1': '*'}
{'surface': '。', 'base': '。', 'pos': '記号', 'pos1': '句点'}


## 31. 動詞 

In [18]:
data_list = create_datalist()
varb_set = set()

for sentence in data_list:
    for word in sentence:
        if word["pos"] == "動詞":
            varb_set.add(word["surface"])

for v in list(varb_set)[:30]:
    print(v)

追っかけ
怒
抜こ
ひやかさ
叩き込み
捲く
行き過ぎ
絶っ
食え
参らせる
寝転ぶ
磨り潰し
断わる
聞き糺し
うなされる
つかっ
しろ
睡ら
っ
過ぎ去っ
隠し
制する
打ち返し
拭う
動く
突い
棄てる
投げ込ん
名づける
似合わ


## 32. 動詞の原型

In [19]:
data_list = create_datalist()
varb_set = set()

for sentence in data_list:
    for word in sentence:
        if word["pos"] == "動詞":
            varb_set.add(word["base"])

for v in list(varb_set)[:30]:
    print(v)

みる
並べ立てる
装う
捲く
済す
寝転ぶ
参らせる
断わる
うなされる
肥る
飛ばす
おくれる
制する
拭う
動く
棄てる
名づける
擦り付ける
寄り添う
開け放つ
併せる
そぐ
行ける
わかす
はぐる
洗う
病む
ふざける
活ける
もつ


## 33. 二つの名詞が「の」で連接されている名詞句

In [20]:
data_list = create_datalist()
nounblock_list = []

for sentence in data_list:
    temp_noun = ""
    for i in range(2, len(sentence)):
        if sentence[i-2]["pos"]=="名詞" and sentence[i-1]["surface"]=="の" and sentence[i]["pos"]=="名詞":
            nounblock_list.append(sentence[i-2]["surface"] + sentence[i-1]["surface"] + sentence[i]["surface"])

for nb in nounblock_list[:30]:
    print(nb)

彼の掌
掌の上
書生の顔
はずの顔
顔の真中
穴の中
書生の掌
掌の裏
何の事
肝心の母親
藁の上
笹原の中
池の前
池の上
一樹の蔭
垣根の穴
隣家の三
時の通路
一刻の猶予
家の内
彼の書生
以外の人間
前の書生
おさんの隙
おさんの三
胸の痞
家の主人
主人の方
鼻の下
吾輩の顔


## 34. 名詞の連接
連続して出現する名詞を最長一致で出力する

In [39]:
data_list = create_datalist()
noungroup_list = []

for sentence in data_list:
    temp_nounlist = []
    for word in sentence:
        if word["pos"]=="名詞":
            temp_nounlist.append(word["surface"])
        else:
            if len(temp_nounlist) >= 2:
                noungroup_list.append("".join(temp_nounlist))
            temp_nounlist = []
    if len(temp_nounlist) >= 2:
        noungroup_list.append("".join(temp_nounlist))
    temp_nounlist = []

for nb in noungroup_list[:30]:
    print(nb)

人間中
一番獰悪
時妙
一毛
その後猫
一度
ぷうぷうと煙
邸内
三毛
書生以外
四五遍
この間おさん
三馬
御台所
まま奥
住家
終日書斎
勉強家
勉強家
勤勉家
二三ページ
主人以外
限り吾輩
朝主人
一番心持
二人
一つ床
一人
最後大変
——猫
